In [ ]:
# 크롤링에 필요한 library 설치
!pip install --upgrade setuptools
!CFLAGS='-stdlib=libc++' pip install konlpy
!pip install selenium

In [ ]:
# 크롤링에 필요한 library import
import time
from konlpy.tag import Okt
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [ ]:
# 크롤링 할 URL 입력
url = "https://www.instagram.com/explore/tags/scottishfold"

# Chrome webdriver를 통해, 크롬 브라우저를 간접적으로 실행 및 조작
# 다운로드 받은 Chrome webdriver 경로 입력
driver = webdriver.Chrome('/Users/mijin/chromedriver')

# 웹 페이지가 완전히 로딩될 수 있도록, 5초간 대기
driver.implicitly_wait(5)

# 크롬 브라우저 실행 및 해당 url로 이동
driver.get(url)

In [ ]:
# 총 게시물 수를 클래스 이름으로 검색 후 저장
totalCount =  driver.find_element_by_class_name('g47SY').text

# 총 게시물 수 출력
print("총 게시물 수:", totalCount)

In [ ]:
# body 태그에 속한 element들을 저장
elem = driver.find_element_by_tag_name("body") 

# 각 게시물의 실제 URL 경로를 담을 빈 리스트 생성
hrefList = []

# 페이지 스크롤을 위한 변수 생성
pageDowns = 1

In [ ]:
# 스크롤을 20번 내리면서 각 게시물의 실제 URL 경로 값 수집
while pageDowns < 20:
        # 스크롤 내림
        elem.send_keys(Keys.PAGE_DOWN)
        
        # 다음 페이지 로딩을 위한 대기
        time.sleep(1)
        
        # 현재 보이는 모든 게시물의 a 태그 값을 저장
        aTags = driver.find_elements_by_css_selector('div.v1Nh3 > a')
        
        # 수집한 a 태그에서 게시물의 실제 URL 경로 값을 추출해 저장
        for a in aTags:
            hrefList.append(a.get_attribute('href'))
            
        # 페이지 스크롤을 1번 내렸다는 의미로 값 증가
        pageDowns += 1

In [ ]:
# 중복된 값을 제거하기 위해 list를 set으로 변환 후 다시 저장
hrefList = list(set(hrefList))

# (Key: Hashtag 이름, Vaule: 출현 빈도)를 한 쌍으로 갖는 dictionary 생성
dictHashtags = {}

# Text에서 hashtag만 추출하기 위해 Okt() 클래스 사용
okt = Okt()

In [ ]:
# 게시물들의 실제 URL 경로로 이동해, hashtag 수집
for href in hrefList:
    # 게시물 URL로 이동
    driver.get(href)
    
    # 게시물 내용(이미지가 아닌)을 담은 부분을 따로 저장
    try: 
        contents = driver.find_element_by_class_name('C4VMK')
    except NoSuchElementException:
        continue
    
    # Text 안의 각 단어 품사 태깅
    text = okt.pos(contents.text)
    
    # 만약 품사가 hashtag로 판명이 되면, dictionary에 추가하고 출현 빈도를 1 증가
    for data in text:
        if data[1] == "Hashtag":
            if not (data[0] in dictHashtags):
                dictHashtags[data[0]] = 1
            else:
                dictHashtags[data[0]] += 1

In [ ]:
# 출현 빈도를 기반으로 dictionary 정렬
keys = sorted(dictHashtags.items(), key = lambda x:x[1], reverse = True)

# 출현 빈도 1~15위 출력
for k, v in keys[:15]:
    print("{}({})".format(k, v))          

In [ ]:
# Chrome webdriver 종료
driver.close() 

In [ ]:
# 시각화에 필요한 library 설치
!pip install wordcloud

In [ ]:
# 시각화에 필요한 library import
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# macos
font_path = '/Library/Fonts/AppleGothic.ttf'

# TODO: 아까 수집한 hashtag와 출현 빈도를 기반으로 Wordcloud 생성
wordcloud = WordCloud(font_path = font_path, background_color="white", width=1000, height=1000, max_words=50, relative_scaling=0.5, normalize_plurals=False).generate_from_frequencies(dictHashtags)

In [ ]:
# Wordcloud를 그려서 저장
fig = plt.figure()
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.savefig('hashtag-wordcloud.png')